# Detection

In [ ]:
# Todo
import torch
from torch.utils.data import DataLoader, TensorDataset
from torchvision import transforms

### Load data and preprocessing

In [3]:
# TODO
# Bilde str 
H_in, W_in = 48, 60
# definere grid str 
H_out, W_out = 2, 3 

# laster så inn den ubehandla dataen
# prøver å gjøre oppgave 3.11 selv om den bre er verdt 1p 
y_true_train = torch.load("navn")
y_true_val = torch.load("navn")
y_true_test = torch.load("navn")


### Normalize Images

### Training

In [ ]:
# TODO


### Prediction

In [ ]:
# TODO

### Model selection and evaluation

In [4]:
def compute_IoU_detection(model, loader, preprocessor):
    """
    Compute IoU performance of the model on the given dataset
    """
    # TODO

def compute_accuracy_detection(model, loader, preprocessor):
    """
    Compute accuracy performance of the model on the given dataset
    """
   # tODO